<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/bgRemoval/tests/PaddleSeg_matting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PaddleSeg - matting + BG-inpainting

[GitHub](https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.5/Matting)


## 1. Preparations

Before start, make sure that you choose

Runtime Type = Python 3
Hardware Accelerator = GPU

In [1]:
!nvidia-smi

Mon May  2 11:50:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. linking nextcloud

Connecting to the external NextCloud drive 

In [2]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='bgRemoval',library='PaddleSeg')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2480  100  2480    0     0  17841      0 --:--:-- --:--:-- --:--:-- 17841
what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## 3. clone GIT repo

In [3]:
import os
root_path = '/content/PaddleSeg'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/PaddlePaddle/PaddleSeg {root_path}

%ls

Cloning into '/content/PaddleSeg'...
remote: Enumerating objects: 17638, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 17638 (delta 53), reused 92 (delta 35), pack-reused 17518
Receiving objects: 100% (17638/17638), 341.81 MiB | 31.76 MiB/s, done.
Resolving deltas: 100% (11275/11275), done.
database/  database_mod.py  PaddleSeg/  __pycache__/  sample_data/


## 4. Setting up the environment

In [4]:
# installing PaddleSeg
%cd {root_path}
!pip install -r requirements.txt
!pip install -e .


# installing Matting
%cd Matting
!pip install -r requirements.txt

/content/PaddleSeg
     |████████████████████████████████| 596 kB 5.4 MB/s 
     |████████████████████████████████| 2.7 MB 34.0 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 
     |████████████████████████████████| 2.1 MB 34.7 MB/s 
     |████████████████████████████████| 197 kB 45.6 MB/s 
     |████████████████████████████████| 69 kB 7.7 MB/s 
     |████████████████████████████████| 42 kB 892 kB/s 
     |████████████████████████████████| 2.0 MB 37.3 MB/s 
     |████████████████████████████████| 8.8 MB 35.9 MB/s 
     |████████████████████████████████| 98 kB 7.1 MB/s 
     |████████████████████████████████| 461 kB 46.8 MB/s 
  Created wheel for bce-python-sdk: filename=bce_python_sdk-0.8.64-py3-none-any.whl size=202973 sha256=9c8123c6184f1a2dfe37808fee7f2a90381bf7469280ed4a72ea831047e2fe73
  Stored in directory: /root/.cache/pip/wheels/fd/ee/a5/4ad3bdc0e60b48e892e8bd6f661a3201d7e76dccfa9e968b34
Successfully built 

## 5. downloading the dataset

In [5]:
%ls

bg_replace.py  dataset/   metric.py   README_CN.md      tools/         utils/
configs/       deploy/    model/      README.md         train.py       val.py
core/          export.py  predict.py  requirements.txt  transforms.py


In [11]:
import os
import zipfile

# define folders
dataset_path = root_path + '/Matting/dataset'
dl_path = 'https://paddleseg.bj.bcebos.com/matting/datasets/PPM-100.zip'

# make folders
os.makedirs(dataset_path, exist_ok=True)

# dowload the dataset
# https://paddleseg.bj.bcebos.com/matting/datasets/PPM-100.zip
print('\n> Download the dataset')
!wget -P {dataset_path} {dl_path}

#!unzip "/content/drive/path/input_file_name.zip" -d "/content/drive/path/output_folder/"
data_src = root_path + '/Matting/dataset/PPM-100.zip'

!unzip -q {data_src}


%ls


> Download the dataset
--2022-05-02 12:26:46--  https://paddleseg.bj.bcebos.com/matting/datasets/PPM-100.zip
Resolving paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263089716 (251M) [application/octet-stream]
Saving to: ‘/content/PaddleSeg/Matting/dataset/PPM-100.zip’

PPM-100.zip         100%[===================>] 250.90M  12.6MB/s    in 45s     

2022-05-02 12:27:33 (5.60 MB/s) - ‘/content/PaddleSeg/Matting/dataset/PPM-100.zip’ saved [263089716/263089716]

bg_replace.py  dataset/   metric.py   README_CN.md      tools/         utils/
configs/       deploy/    model/      README.md         train.py       val.py
core/          export.py  predict.py  requirements.txt  transforms.py
